Se importan los paquetes necesarios

In [1]:
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("src")
import cleaning_functions as cf

Se importan los datos como pandas DataFrame

In [3]:
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

Se exploran los datos

In [4]:
sharks.shape

(25723, 24)

In [5]:
sharks.sample(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
8477,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4077,1959.08.05,05-Aug-1959,1959.0,Invalid,USA,California,"Hermosa Beach, Los Angeles County",Wading,Charles H. Hill,M,...,Invalid,"Independent, 8/6/1959, p.8; H.D. Baldridge (19...",1959.08.05-Hill.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1959.08.05,1959.08.05,2226.0,NaN,NaN
3836,1962.02.05,05-Feb-1962,1962.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Winkelspruit,Treading water,Reece Nielsen,M,...,"White shark, 3 m","M. Nielsen, M. Levine, GSAF; D. Davies, p.115",1962.02.05-Nielsen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1962.02.05,1962.02.05,2467.0,NaN,NaN
18894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3287,1973.12.19.R,Reported 18-Dec-1973,1973.0,Unprovoked,PHILIPPINES,Luzon,NaN,Swimming,Ron Arney,M,...,NaN,"Washington Post, 12/19/1973",1973.12.19.R-Arney.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1973.12.19.R,1973.12.19.R,3016.0,NaN,NaN


In [6]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [7]:
sharks.isna().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

Se comprueba que es un DataFrame muy sucio con una gran cantidad de nulos

Se cambia el nombre de todas las columnas del DataFrame para simplificar

In [8]:
nuevas_columna = {columna:(columna.replace(" ", "_").replace(".", "_").lower() if columna[-1]!=" "
                  else columna.replace(" ", "_").replace(".", "_").lower()[:-1]) for columna in list(sharks.keys())}

In [9]:
sharks.rename(columns=nuevas_columna, inplace=True)

In [10]:
sharks.sample(5)

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
19385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6467,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,2016.04.18.b,18-Apr-2016,2016.0,Provoked,FRENCH POLYNESIA,Tuamotos,Makemo Atoll,Spearfishing,Hoata Iotua,M,...,"Grey reef shark, 2 m","TNTV News, 4/19/2016",2016.04.18.b-Iotua.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.04.18.b,2016.04.18.b,6018.0,NaN,NaN
8099,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ahora se va a proceder a la limpieza de los datos.

Lo primero es eliminar todas las filas con datos repetidos

In [11]:
sharks.drop_duplicates(inplace=True)

A continuación se van a eliminar todas aquellas filas donde todas las celdas sean nulos y todas las columnas donde todos los valores sean nulos

In [12]:
sharks.dropna(axis=0, how='all', inplace=True)
sharks.dropna(axis=1, how='all', inplace=True)

In [13]:
sharks.shape

(6311, 24)

Con este proceso se han limpiado bastantes filas.

In [14]:
sharks[(sharks["case_number"]=="0")].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 6302 to 6309
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8 non-null      object 
 1   date                    0 non-null      object 
 2   year                    0 non-null      float64
 3   type                    0 non-null      object 
 4   country                 0 non-null      object 
 5   area                    0 non-null      object 
 6   location                0 non-null      object 
 7   activity                0 non-null      object 
 8   name                    0 non-null      object 
 9   sex                     0 non-null      object 
 10  age                     0 non-null      object 
 11  injury                  0 non-null      object 
 12  fatal_(y/n)             0 non-null      object 
 13  time                    0 non-null      object 
 14  species                 0 non-null      

In [15]:
sharks[(sharks["case_number"]=="0")].shape

(8, 24)

Se comprueba que en aquellas columnas donde "case_number" es nulo, todo el resto de columnas salvo "original__order" son nulas también, por lo que no aportan información útil y se van a eliminar.

In [16]:
sharks.drop(sharks[(sharks["case_number"]=="0")].index,inplace= True)

In [17]:
sharks.isna().sum()

case_number                  1
date                         1
year                         3
type                         5
country                     51
area                       456
location                   541
activity                   545
name                       211
sex                        566
age                       2832
injury                      29
fatal_(y/n)                540
time                      3355
species                   2839
investigator_or_source      18
pdf                          1
href_formula                 2
href                         1
case_number_1                1
case_number_2                1
original_order               1
unnamed:_22               6302
unnamed:_23               6301
dtype: int64

Se comprueba que el DataFrame está mucho más limpio. Viendo la información del DF se ve que las columnas "unnamed:_22" y "unnamed:_23" casi todos los valores son NaN. Se va a verificar cuales son aquellos valores no nulos y posteriormente eliminar las columnas completas

In [18]:
sharks.shape

(6303, 24)

In [19]:
sharks[sharks["unnamed:_22"].notna()].sample()

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
1478,2006.05.27,27-May-2006,2006.0,Unprovoked,USA,Hawaii,"North Shore, O'ahu",Surfing,Bret Desmond,M,...,NaN,R. Collier,2006.05.27-Desmond.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.05.27,2006.05.27,4825.0,stopped here,NaN


In [20]:
sharks[sharks["unnamed:_23"].notna()].sample(2)

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


No hay ningún dato valioso en esas columnas por lo que se van a eliminar

In [21]:
sharks.drop(columns = ["unnamed:_22", "unnamed:_23"],axis=1,inplace= True)

In [22]:
sharks.sample(5)

,case_number,date,year,type,country,area,location,activity,name,sex,...,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order
3916,1961.03.09,09-Mar-1961,1961.0,Unprovoked,NORTH PACIFIC OCEAN,Wake Island (EnenKio),"Leeward side of island, directly in back of Mi...",Free diving,James K. Stewart,M,...,N,15h30,"Grey reef shark, 1.8 m [6'] grey reef shark, i...",J.K. Stewart; Albert Tester; V.M. Coppleson (1...,1961.03.09-Stewart.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1961.03.09,1961.03.09,2387.0
6000,1855.09.30,30-Sep-1855,1855.0,Sea Disaster,USA,North Carolina,Cape Hatteras,ship William Penn grounded & broke apart,sailor,M,...,N,NaN,NaN,"NY Times, 10/12/1855",1855.09.30-WilliamPenn.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1855.09.30,1855.09.30,303.0
2899,1984.03.10,10-Mar-1984,1984.0,Boat,SOUTH AFRICA,Eastern Cape Province,Swartkops River mouth,Rowing,"rowboat, occupant: Louis Beyers",M,...,N,NaN,2.5 m [8.25'] shark,"M. Levine, GSAF",1984.03.10-Beyers.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1984.03.10,1984.03.10,3404.0
1619,2004.12.16,16-Dec-2004,2004.0,Unprovoked,AUSTRALIA,South Australia,"West Beach, Adelaide",Scurfing (surfboard being towed behind a boat),Nick Peterson,M,...,Y,15h15,4.5 m & 5 m white shark,"P. Kemp & T. Peake, GSAF",2004.12.16-Peterson-draft.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2004.12.16,2004.12.16,4684.0
81,2017.09.20,20-Sep-2017,2017.0,Unprovoked,USA,Florida,Palm Beach County,Surfing,Richard Inniss,M,...,N,Morning,NaN,"WPTV, 9/25/2017",2017.09.20-Inniss.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.20,2017.09.20,6222.0


In [23]:
sharks.shape

(6303, 22)

In [24]:
sharks[sharks["case_number_1"]!=sharks["case_number_2"]].sample(21)[["case_number","case_number_1","case_number_2"]]

,case_number,case_number_1,case_number_2
5275,1900.00.00.R,1919.00.00.R,1900.00.00.R
3522,1967.07.05,1967/07.05,1967.07.05
6122,1808.06.01.R,1808.05.01.R,1808.06.01.R
25722,xx,NaN,NaN
144,2017.05.06,2017.06.06,2017.05.06
6009,"1853,94.29",1853.04.29,"1853,94.29"
3880,1961.09.02.R,"1961.09,06.R",1961.09.02.R
117,2017.07.20.a,2017/07.20.a,2017.07.20.a
4403,1952.08.05,1952.08.04,1952.08.05
5358,1913.07.10.R,1913.17.10.R,1913.07.10.R


Viendo la compación de las columnas "case_number_1" y"case_number_2" con "case_number" se ve que la columna "case_number_1" contiene la misma información que la de "case_number_2" salvo en una pequeña cantidad de casos donde la columna "case_number" coincide con "case_number_2" por lo que se asume que la columna "case_number_1" no aporta nada de valor

In [25]:
sharks.drop(columns = ["case_number_1"],axis=1,inplace= True)

In [26]:
sharks[sharks["case_number"]!=sharks["case_number_2"]][["case_number","case_number_2","date"]].sample(5)

,case_number,case_number_2,date
25722,xx,NaN,NaN
4949,1934.01.08.R,1934.02.08.R,Reported 08-Feb-1934
5944,1864.05,1864.05.00,May-1864
5488,NaN,1905.09.06.R,Reported 06-Sep-1905
390,2015.07-10,2015.07.10,10-Jul-2015


Se comprueba que las columnas "case_number" y "case_number_2" son iguales salvo en unos casos donde comprobando con la columna date se ve que es la columna "case_number" la que es erronea

In [27]:
sharks.drop(columns = ["case_number"],axis=1,inplace= True)

In [28]:
sharks.sample(5)

,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_2,original_order
6129,Mar-1803,1803.0,Unprovoked,AUSTRALIA,Western Australia,"Hamelin Harbour, at Faure Island",NaN,M. Lefevre & a sailor (rescuer),M,NaN,Shark knocked him down & tore clothing of the ...,N,NaN,NaN,F. Peron ref in G.P. Whitley (Fishes of Austr...,1803.03.00-Lefevre.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1803.03.00,174.0
3382,13-Sep-1970,1970.0,Provoked,PALAU,Western Caroline Islands (North Pacific Ocean),Outside barrier reef,Spearfishing,Aismerael Samsel,M,20,"Another diver shot shark, shark bit his left f...",N,NaN,1 m shark,"K.R.H. Read; H.D. Baldridge, p.192",1970.09.13-Samsel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.09.13,2921.0
4181,31-Dec-1957,1957.0,Unprovoked,AUSTRALIA,South Australia,"Port Hughes, 100 miles from Adelaide in Spence...",Spearfishing,Jack Evans,M,NaN,No injury. Shark grabbed fish attached to his ...,N,NaN,NaN,"V.M. Coppleson (1962), p.251",1957.12.31-Evans.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.12.31,2122.0
6000,30-Sep-1855,1855.0,Sea Disaster,USA,North Carolina,Cape Hatteras,ship William Penn grounded & broke apart,sailor,M,NaN,Foot bitten,N,NaN,NaN,"NY Times, 10/12/1855",1855.09.30-WilliamPenn.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1855.09.30,303.0
5390,01-May-1911,1911.0,Unprovoked,SOUTH AFRICA,Western Cape Province,Victoria Bay,Swimming,James Jantjes,M,NaN,FATAL,Y,NaN,NaN,"Cape Mercantile Advertiser, 5/2/1911, M. Levin...",1911.05.01-Jantjies.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1911.05.01,913.0


In [29]:
sharks.isna().sum()

date                         1
year                         3
type                         5
country                     51
area                       456
location                   541
activity                   545
name                       211
sex                        566
age                       2832
injury                      29
fatal_(y/n)                540
time                      3355
species                   2839
investigator_or_source      18
pdf                          1
href_formula                 2
href                         1
case_number_2                1
original_order               1
dtype: int64

Hasta aquí se han limpiado del DataFrame todas aquellas filas con todos los valores NaN y todas aquellas columnas que no aportan nada de información útil, a continuación se va a exportar el DataFrame modificado a un .csv

In [30]:
sharks.to_csv("data/sharks_limpio_raw.csv",index=False)

In [49]:
sharks = pd.read_csv("data/sharks_limpio_raw.csv",encoding = "ISO-8859-1")

Se van a eliminar todas aquellas columnas que no se van a usar para sacar conclusiones

In [50]:
sharks.drop(columns = ["investigator_or_source","pdf","href_formula","href","case_number_2","original_order"],axis=1,inplace= True)

In [51]:
sharks

,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6298,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN
6299,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN
6300,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ÂºN, 79ÂºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN
6301,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN


Ahora se va a hacer una limpieza de cada columna útil

Se empieza por la columna "sex"

In [32]:
list(sharks["sex"].unique())

['F', 'M', nan, 'M ', 'lli', 'N', '.']

In [33]:
sharks["sex"] = sharks["sex"].apply(cf.limpieza_sex)

In [34]:
sharks["sex"].unique()

array(['F', 'M', None], dtype=object)

Se continua por la columna "age"

In [35]:
print(list(sharks["age"].unique()))

['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21', '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25', '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24', '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8', '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen', '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6', '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53', '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20', '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26', '8 or 10', '84', 'Â\xa0 ', ' ', '30 or 36', '6Â½', '21 & ?', '75', '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28', '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35', '23 & 26', '1', '(adult)', '33 & 37', '25 or 28', '37, 67, 35, 27,  ? & 27', '21, 34,24 & 35', '3

In [36]:
sharks["age"] = sharks["age"].apply(cf.limpieza_age)

In [37]:
sharks["age"].unique()

array([57, 11, 48, 'unknown', 18, 52, 15, 12, 32, 10, 21, 34, 30, 60, 33,
       29, 54, 41, 37, 56, 19, 25, 69, 38, 55, 35, 46, 45, 14, 40, 28, 20,
       24, 26, 49, 22, 7, 31, 17, 13, 42, 3, 8, 50, 16, 82, 73, 68, 51,
       39, 58, 47, 61, 65, 36, 66, 43, 9, 72, 59, 6, 27, 64, 23, 71, 44,
       62, 63, 70, 1, 53, 77, 74, 5, 86, 84, 75, 87, 67, 0, 81, 78, 2],
      dtype=object)

Ahora la columna "year"

In [38]:
sharks["year"].unique()

array([2018., 2017.,   nan, 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [39]:
sharks["year"] = sharks["year"].apply(cf.limpieza_year)

In [40]:
sharks["year"].unique()

array([2018, 2017, 'unknown', 2016, 2015, 2014, 2013, 2012, 2011, 2010,
       2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999,
       1998, 1997, 1996, 1995, 1984, 1994, 1993, 1992, 1991, 1990, 1989,
       1969, 1988, 1987, 1986, 1985, 1983, 1982, 1981, 1980, 1979, 1978,
       1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1968, 1967, 1966,
       1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955,
       1954, 1953, 1952, 1951, 1950, 1949, 1948, 1848, 1947, 1946, 1945,
       1944, 1943, 1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934,
       1933, 1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923,
       1922, 1921, 1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912,
       1911, 1910, 1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901,
       1900, 1899, 1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891, 1890,
       1889, 1888, 1887, 1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879,
       1878, 1877, 1876, 1875, 1874, 1873, 1872, 187

Ahora la columna "fatal_(y/n)"

In [41]:
sharks["fatal_(y/n)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [42]:
sharks["fatal_(y/n)"] = sharks["fatal_(y/n)"].apply(cf.limpieza_fatal)

In [43]:
sharks["fatal_(y/n)"].unique()

array(['NO', 'YES', 'unknown'], dtype=object)

Ahora la columna time

In [52]:
sharks["time"].unique()

array(['18h00', '14h00  -15h00', '07h45', nan, 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '15h30', '07h30', '19h00, Dusk', 'Night', '16h00',
       '15h01', '12h00', '13h45', '23h30', '09h00', '14h30', '18h30',
       '12h30', '16h30', '18h45', '06h00', '10h00', '10h44', '13h19',
       'Midday', '13h30', '10h45', '11h20', '11h45', '19h30', '08h30',
       '15h45', 'Shortly before 12h00', '17h34', '17h10', '11h15',
       '08h50', '17h45', '13h00', '10h20', '13h20', '02h00', '09h50',
       '11h30', '17h30', '9h00', '10h43', 'After noon', '15h15', '15h40',
       '19h05', '1300', '14h30 / 15h30', '22h00', '16h20', '14h34',
       '15h25', '14h55', '17h46', 'Morning ', '15h49', '19h00',
       'Midnight', '09h30 / 10h00', '10h15', '18h15', '04h00', '14h50',
       '13h50', '19h20', '10h25', '10h45-11h15', '16h45', '15h52',
       '06h15', '14h